In [2]:
import requests
import lxml.html as lh
import pandas as pd

In [3]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, wikipedia_page, to handle the contents of the website
wikipedia_page=requests.get(wikipedia_link)

#Storing the contents of the website under wikipedia_page
wikipedia_page = lh.fromstring(wikipedia_page.content)

#Parsing data that are stored between <tr>..</tr> of HTML
tr_elements = wikipedia_page.xpath('//tr')

#Checking the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [4]:
tr_elements = wikipedia_page.xpath('//tr')

#Create empty list
col=[]

i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print(name)
    col.append((name,[]))

Postal code

Borough

Neighborhood



In [5]:
col

[('Postal code\n', []), ('Borough\n', []), ('Neighborhood\n', [])]

In [12]:
#Since out first row is the header, data is stored on the second row onwards
for k in range(1,len(tr_elements)):
    #T is our k'th row
    T=tr_elements[k]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Checking if row is empty
        if i>0:
        #Converting any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Appending the data to the empty list of the i'th column
        col[i][1].append(data)
        #Incrementing i for the next column
        i+=1

In [13]:
[len(C) for (title,C) in col]

[362, 362, 362]

In [14]:
# Transforming the dictionary on a dataframe
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [15]:
# Cleaning break lines
df = df.replace(r'\n','', regex=True)
df.columns = ['Postcode', 'Borough', 'Neighbourhood']
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [16]:
#Removing and fixing df index
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)
df.index = range(len(df))
df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [17]:
df.shape

(208, 3)

In [22]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [23]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

In [24]:
#Find the first table on the Wikipedia page and iterate through tags for required information
ZipCode = soup.find('table')
colvals = ZipCode.find_all('td')

elem_count = len(colvals)

postcode = []
borough = []
neighborhood = []

for i in range(0, elem_count, 3):
    postcode.append(colvals[i].text.strip())
    borough.append(colvals[i+1].text.strip())
    neighborhood.append(colvals[i+2].text.strip())

In [27]:
df_postcode = pd.DataFrame(data=[postcode, borough, neighborhood]).transpose()
df_postcode.columns = ['Postcode', 'Borough', 'Neighborhood']

In [28]:
df_postcode.drop(df_postcode[df_can_postcode['Borough'] == 'Not assigned'].index, inplace=True)
df_postcode.loc[df_postcode.Neighborhood == 'Not assigned', "Neighborhood"] = df_postcode.Borough

In [29]:
df_grp_can = df_can_postcode.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_grp_can.columns = ['Postcode', 'Borough', 'Neighborhood']

In [30]:
df_latitudeLong = pd.read_csv('http://cocl.us/Geospatial_data')
df_latitudeLong.columns = ['Postcode', 'Latitude', 'Longitude']

In [31]:
df_join = pd.merge(df_grp_can, df_latitulng, on=['Postcode'], how='inner')

NameError: name 'df_latlng' is not defined